In [291]:
import praw
import re
from collections import Counter
from collections import defaultdict
from itertools import chain

In [38]:
APP_ID = 'ACqyZR3hYsXzfA'
APP_SECRET = 'xehsNEoOe_EH9DT-JwxekUDXLKxOIA'
USER_AGENT = 'my little scrapper'

In [39]:
reddit = praw.Reddit(
     client_id=APP_ID,
     client_secret=APP_SECRET,
     user_agent=USER_AGENT
)

In [40]:
subreddit = reddit.subreddit("wallstreetbets")

In [279]:
print(subreddit.display_name)

wallstreetbets


In [42]:
hot_submissions = []

for submission in subreddit.hot(limit=10):
    author_name = submission.author.name
    author_redditor_instance = reddit.redditor(author_name) 
    new_sumbission = {
        'title': submission.title,  # Output: the submission's title
        'score': submission.score,  # Output: the submission's score
        'submission_id': submission.id,     # Output: the submission's ID
        'submission_url': submission.url,   # Output: the URL the submission points to
        'author': {
            'name': author_name,
            'comment_karma': author_redditor_instance.comment_karma
        }
    }
    hot_submissions.append(new_sumbission)

In [224]:
all_comments = []

url = hot_submissions[0]['submission_url']
sub = reddit.submission(url=url)

for hot_sub in hot_submissions:
    hot_sub_url = hot_sub['submission_url']
    if hot_sub_url.endswith("/"):
        sub = reddit.submission(url=hot_sub['submission_url'])
        for comment in sub.comments:
            if hasattr(comment, 'body'):
                author_name = comment.author.name if comment.author else None
                
                if author_name:
                    redditor = reddit.redditor(author_name)
                    comment_karma = redditor.comment_karma if redditor.comment_karma else None
                
                num_of_replies = 0
                if (len(comment.replies._comments) > 0):
                    num_of_replies = comment.replies._comments[0].count if hasattr(comment.replies._comments[0], 'count') else 0
                all_comments.append({
                    'body': comment.body,
                    'author': author_name,
                    'author_comment_karma': comment_karma,
                    'ups': comment.ups,
                    'downs': comment.downs,
                    'awards_count': comment.total_awards_received,
                    'created_at': comment.created,
                    'num_of_replies': num_of_replies
                })

In [286]:
# first version, assumes you do not know the stock name
# uses regex to filter comment for exaclty three capital letters in a row
# with a space or a dollar sign before and a space after
# dumb bc returns too many common words

stock_name_re = '( [A-Z]{3} | \$[A-Z]{3} )'

stock_mentions = {}

for com in all_comments:
    com_stocks_names = re.findall(stock_name_re, com['body'])
    if len(com_stocks_names) > 0:
        for com_stock_name in com_stocks_names:
            name = com_stock_name.strip()
            if name[0] == '$':
                name = name[1:]
            if name in stock_mentions:
                stock_mentions[name] += 1
            else:
                stock_mentions[name] = 1

In [304]:
# second version
# uses a list with stock names
# and Counters to count their occurencs in comments

stocks_to_look_for = ['GME', 'AMC']

res = {
    'GME': 0,
    'AMC': 0
}

for com in all_comments:
    text = com['body']
    counter = Counter(w for w in text.split() if w in stocks_to_look_for)
    if len(counter) >0:
        if 'GME' in counter:
            res['GME'] += counter['GME']
        if 'AMC' in counter:
            res['AMC'] += counter['AMC']

res

{'GME': 171, 'AMC': 93}